In [ ]:
import pandas as pd
import holidays
import locale

# Définir la locale en français
locale.setlocale(locale.LC_TIME, 'fr_FR')
# Définir la plage de dates
start_date = '2021-01-01'
end_date = '2023-06-01'


# Obtenir les jours fériés pour la France
fr_holidays = holidays.France()

# Liste des jours fériés entre les dates spécifiées
jours_feries = pd.to_datetime([date for date in fr_holidays[start_date:end_date]])

In [ ]:
df_combinations = pd.MultiIndex.from_product([ pd.date_range(start_date, end_date)])
df_data = pd.DataFrame(index=df_combinations).reset_index(names='date')


# Ajouter une colonne avec le nom du jour en français
df_data['jour'] = df_data['date'].dt.strftime('%d').astype(int)

df_data['joursemaine'] = df_data['date'].dt.dayofweek

df_data['mois'] = df_data['date'].dt.month

df_data['annee'] = df_data['date'].dt.year

df_data['semaine'] = df_data['date'].dt.strftime('%W').astype(int)


# df_all_data.iloc[80:100,:]
df_data['Ferie']=0
# Vérifier si la date est un jour férié et ajouter un label correspondant
df_data.loc[df_data['date'].isin(jours_feries), 'Ferie'] = 1


# Créer une liste pour les jours précédents et suivants les jours fériés
jours_precedents = [date - pd.DateOffset(days=1) for date in df_data['date'] if date in jours_feries]
jours_suivants = [date + pd.DateOffset(days=1) for date in df_data['date'] if date in jours_feries]
    
# Ajouter les labels pour les jours précédents et suivants dans la colonne "jour"

df_data['avant_Ferie']=0
df_data['après_Ferie']=0
df_data.loc[df_data['date'].isin(jours_precedents), 'avant_Ferie'] = 1
df_data.loc[df_data['date'].isin(jours_suivants), 'après_Ferie'] = 1

df_data

,date,jour,joursemaine,mois,annee,semaine,Ferie,avant_Ferie,après_Ferie
0,2021-01-01,1,4,1,2021,0,1,0,0
1,2021-01-02,2,5,1,2021,0,0,0,1
2,2021-01-03,3,6,1,2021,0,0,0,0
3,2021-01-04,4,0,1,2021,1,0,0,0
4,2021-01-05,5,1,1,2021,1,0,0,0
...,...,...,...,...,...,...,...,...,...
877,2023-05-28,28,6,5,2023,21,0,1,0
878,2023-05-29,29,0,5,2023,22,1,0,0
879,2023-05-30,30,1,5,2023,22,0,0,1
880,2023-05-31,31,2,5,2023,22,0,0,0


## Gestion jours soldes 

In [ ]:
import pandas as pd
import datetime

# Définir les dates de début et de fin des soldes d'hiver pour chaque année
soldes_hiver_dates = {
    2019: (datetime.date(2019, 1, 9), datetime.date(2019, 2, 19)),
    2020: (datetime.date(2020, 1, 8), datetime.date(2020, 2, 4)),
    2021: (datetime.date(2021, 1, 20), datetime.date(2021, 2, 16)),
    2022: (datetime.date(2022, 1, 12), datetime.date(2022, 2, 8)),
    2023: (datetime.date(2023, 1, 11), datetime.date(2023, 2, 7))
}

# Définir les dates de début et de fin des soldes d'été pour chaque année
soldes_ete_dates = {
    2019: (datetime.date(2019, 6, 26), datetime.date(2019, 8, 6)),
    2020: (datetime.date(2020, 7, 15), datetime.date(2020, 8, 11)),
    2021: (datetime.date(2021, 6, 30), datetime.date(2021, 7, 27)),
    2022: (datetime.date(2022, 6, 22), datetime.date(2022, 7, 19)),
    2023: (datetime.date(2023, 6, 28), datetime.date(2023, 7, 25))
}
def label_solde_ete(row):
    date = row['date']
    year = date.year
    debut_ete, fin_ete = soldes_ete_dates.get(year, (None, None))

    if debut_ete and fin_ete and debut_ete <= date <= fin_ete:
        delta = (date - pd.Timestamp(debut_ete)).days
        semaine_solde = delta // 7  # Diviser le nombre de jours par 7 pour obtenir le numéro de la semaine
        row['solde_ete'] = semaine_solde 
    else:
        row['solde_ete'] = 0
    return row

# Fonction pour attribuer le label de la semaine de solde
def label_solde_hiver(row):
    date = row['date']
    year = date.year
    debut_hiver, fin_hiver = soldes_hiver_dates.get(year, (None, None))

    if debut_hiver and fin_hiver and debut_hiver <= date <= fin_hiver:
        delta = (date - pd.Timestamp(debut_hiver) ).days
        semaine_solde = delta // 7  # Diviser le nombre de jours par 7 pour obtenir le numéro de la semaine
        row['solde_hiver'] = semaine_solde 
        
    else:
        row['solde_hiver'] = 0
    return row

# Appliquer la fonction à chaque ligne du dataframe
df_data['solde_ete'] = 0
df_data['solde_hiver'] = 0
df_data = df_data.apply(label_solde_hiver, axis=1)
df_data = df_data.apply(label_solde_ete, axis=1)
df_data


/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_11277/1976678666.py:40: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if debut_hiver and fin_hiver and debut_hiver <= date <= fin_hiver:
/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_11277/1976678666.py:26: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if debut_ete and fin_ete and debut_ete <= date <= fin_ete:


,date,jour,joursemaine,mois,annee,semaine,Ferie,avant_Ferie,après_Ferie,solde_ete,solde_hiver
0,2021-01-01,1,4,1,2021,0,1,0,0,0,0
1,2021-01-02,2,5,1,2021,0,0,0,1,0,0
2,2021-01-03,3,6,1,2021,0,0,0,0,0,0
3,2021-01-04,4,0,1,2021,1,0,0,0,0,0
4,2021-01-05,5,1,1,2021,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
877,2023-05-28,28,6,5,2023,21,0,1,0,0,0
878,2023-05-29,29,0,5,2023,22,1,0,0,0,0
879,2023-05-30,30,1,5,2023,22,0,0,1,0,0
880,2023-05-31,31,2,5,2023,22,0,0,0,0,0


In [ ]:
df_data.solde_hiver.value_counts()

0    819
1     21
2     21
3     21
Name: solde_hiver, dtype: int64

In [ ]:
df_data.to_csv('data/ferie_solde.csv', index=False)

In [ ]:
df_tel = pd.read_csv('data/telephonie_clean.csv', index_col="date_appel", parse_dates = True)

df_tel = df_tel.loc[ "2021-01-01":,:]

df = df_tel[(df_tel["Entite"] == 'Entite 1') & (df_tel["Famille"] == 'F2')]
df = df.reset_index(names="date")


In [ ]:
df.merge(df_data,on="date",how="left")

,date,Entite,Famille,Nombre_entrants_corrige,jour,joursemaine,mois,annee,semaine,Ferie,avant_Ferie,après_Ferie,solde_ete,solde_hiver
0,2021-01-02,Entite 1,F2,13.200000,2,5,1,2021,0,0,0,1,0,0
1,2021-01-04,Entite 1,F2,171.637875,4,0,1,2021,1,0,0,0,0,0
2,2021-01-05,Entite 1,F2,139.823805,5,1,1,2021,1,0,0,0,0,0
3,2021-01-06,Entite 1,F2,186.613478,6,2,1,2021,1,0,0,0,0,0
4,2021-01-07,Entite 1,F2,143.625028,7,3,1,2021,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,2023-02-10,Entite 1,F2,62.526316,10,4,2,2023,6,0,0,0,0,0
643,2023-02-11,Entite 1,F2,16.170000,11,5,2,2023,6,0,0,0,0,0
644,2023-02-13,Entite 1,F2,75.822353,13,0,2,2023,7,0,0,0,0,0
645,2023-02-14,Entite 1,F2,65.276712,14,1,2,2023,7,0,0,0,0,0
